In [13]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [14]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [15]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [16]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [17]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [18]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [25]:
class ConnectTimeout(Exception):
    """Custom exception for connection timeout."""
    pass


In [26]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [27]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [28]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
stream_brochure("Leetcode", "https://leetcode.com/")

Found links: {'relevant_links': ['https://leetcode.com/about/', 'https://leetcode.com/jobs/', 'https://leetcode.com/discuss/', 'https://leetcode.com/explore/', 'https://leetcode.com/company/', 'https://leetcode.com/team/']}


# Welcome to LeetCode: Your Favorite Code Playground!

## 🌟 Overview
At LeetCode, we believe that coding should be as fun as a roller coaster ride (minus the motion sickness)! Our mission? To prepare you for the wild world of technical interviews while keeping you entertained with delightful problems and challenges.

## 🤖 Company Culture
At LeetCode, our culture is dominated by lines of code and laughter! We’re a diverse team of tech enthusiasts, caffeine aficionados, and perhaps a few closet mathematicians. Every day is a unique adventure of debugging our lives while simplifying solutions—kind of like cooking oatmeal...more water, less coding mess. 

Our secret ingredient? Team spirit! Whether it’s coding sessions or contests, we thrive on camaraderie and a sprinkle of friendly competition.

## 🏆 Who’s Our Customer?
- Aspiring software engineers looking to crack the coding interview with finesse and possibly an unnecessary amount of angst.
- Veteran developers eager to polish their skills and maybe unlock some coding achievements, like the elusive “Unicorn Developer” badge (note: badge may not actually exist).
- Anyone who has ever thought “I should definitely learn algorithms” but forgot to complete that thought.

## 🚀 Careers at LeetCode
Are you ready to join the coding circus? We have a variety of roles that could fit your dreams and aspirations (no clowns, we promise):
- Software Engineers: Love writing code? Join our team to write code that writes codes about other codes!
- Product Managers: Cultivate that strategic mindset while preventing our user satisfaction rate from becoming a horror story.
- Marketing Mavericks: Help us reach out to the world and demonstrate why coding problems can make for great dinner conversation.

**Do you possess any unusual talents?** Don’t be shy! We want to hear about the quirkiest skills you bring to the table; interpretive dance for code presentations, anyone?

## 💻 Engage with Us
Get involved in our community through forums, problem discussions, and perhaps debate if pineapple belongs on pizza. Join our contests and discover where you stand against other coders around the globe!

### Don't Forget!
Be sure to check out our vast array of programming challenges designed to stretch your brain and make you question your life choices (in a good way, of course).

**Social Media Links**
- [Twitter](https://twitter.com/leetcode)
- [LinkedIn](https://www.linkedin.com/company/leetcode)
- [Facebook](https://www.facebook.com/Leetcode)
- [Instagram](https://www.instagram.com/leetcode)

**Join us at LeetCode, and let’s code our way into a brighter technological future—one problem at a time!** 🚀💻✨